In [1]:
%%capture
%pip install -e ../../..

In [2]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

# Load the .env file
load_dotenv()

# Get the Hugging Face token from the environment variable
hf_token = os.getenv("HF_TOKEN")

# Login using the token
if hf_token:
    login(token=hf_token)
    print("Successfully logged in to Hugging Face")
else:
    print("HUGGINGFACE_TOKEN not found in .env file")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Successfully logged in to Hugging Face


In [3]:
from lmexp.models.implementations.gpt2small import GPT2Tokenizer, ProbedGPT2
from lmexp.models.implementations.llama2 import Llama2Tokenizer, ProbedLlama2
from lmexp.models.implementations.llama3 import Llama3Tokenizer, ProbedLlama3
from lmexp.generic.probing import train_probe
from lmexp.generic.caa import get_caa_vecs
from lmexp.generic.hooked_model import run_simple_steering
from datetime import datetime
import random

# Load GPT2 model and tokenizer

These classes have already implemented all the probing-related methods so we won't have to add more hooks + they are ready to use with our vector extraction and steering functions.

In [ ]:
model = ProbedGPT2()
tokenizer = GPT2Tokenizer()

In [ ]:
model.get_n_layers()

# Load Llama 2 model and tokenizer

Don't run all models at the same time. Select one.

In [ ]:
model = ProbedLlama2()

In [ ]:
tokenizer = Llama2Tokenizer()

In [ ]:
model.get_n_layers()

# Load Llama 3 model and tokenizer

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:
model = ProbedLlama3()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
tokenizer = Llama3Tokenizer()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model.get_n_layers()

32

# Training a linear probe

## Generate some data

Let's see whether we can get a date/time probe vector

In [8]:
def gen_labeled_text(n):
    # date as text, date as utc timestamp in seconds, sample randomly from between 1990 and 2022
    start_timestamp = datetime(2013, 1, 1).timestamp()
    end_timestamp = datetime(2016, 1, 1).timestamp()
    labeled_text = []
    for i in range(n):
        timestamp = start_timestamp + (end_timestamp - start_timestamp) * random.random()
        date = datetime.fromtimestamp(timestamp)
        # date like "Monday 15th November 2021 8AM"
        text = date.strftime("Today is a %A. It's the %dth of %B, %Y. The time is %I %p. This is the point in time when")
        label = timestamp
        labeled_text.append((text, label))
    # normalize labels to have mean 0 and std 1
    labels = [label for _, label in labeled_text]
    mean = sum(labels) / len(labels)
    std = (sum((label - mean) ** 2 for label in labels) / len(labels)) ** 0.5
    labeled_text = [(text, (label - mean) / std) for text, label in labeled_text]
    return labeled_text

In [9]:
data = gen_labeled_text(10_000)
print(data[0])

("Today is a Tuesday. It's the 29th of April, 2014. The time is 04 PM. This is the point in time when", -0.22247807393657132)


## Training

In [ ]:
probe = train_probe(
    labeled_text=data,
    model=model,
    tokenizer=tokenizer,
    layer=16,
    n_epochs=20,
    batch_size=64, #Batch size 128 does not work for Llama 3 8GB at full precision with A100 80GB (64 requires 70GB)
    lr=1e-2,
    save_to=None,
    token_position=0
)

  0%|                                                                                                           | 0/156 [00:00<?, ?it/s]

Input to ProbedLlama3 shape: torch.Size([64, 1, 33])


  1%|▋                                                                                                  | 1/156 [00:02<05:51,  2.27s/it]

Input to ProbedLlama3 shape: torch.Size([64, 1, 33])


  1%|█▎                                                                                                 | 2/156 [00:04<05:03,  1.97s/it]

...

Input to ProbedLlama3 shape: torch.Size([64, 1, 33])


 28%|███████████████████████████▋                                                                      | 44/156 [01:17<03:16,  1.76s/it]

Input to ProbedLlama3 shape: torch.Size([64, 1, 33])


 29%|████████████████████████████▎                                                                     | 45/156 [01:19<03:14,  1.76s/it]

Input to ProbedLlama3 shape: torch.Size([64, 1, 33])


## Using the vector

In [ ]:
direction = probe.weight[0]
bias = probe.bias

In [ ]:
bias

In [ ]:
for multiplier in [-1.0, -0.5, -0.1, -0.05, -0.01, -0.001, 0, 0.001, 0.01, 0.05, 0.1, 0.5, 1.0]:
    result = run_simple_steering(
        text=["The current date is"],
        model=model,
        tokenizer=tokenizer,
        layer=16,
        multiplier=multiplier,
        vector=direction.detach(),
        max_n_tokens=10,
        save_to=None
    )
    if isinstance(result, list) and len(result) > 0:
        if isinstance(result[0], dict) and 'output' in result[0]:
            print(f"Multiplier {multiplier}: {result[0]['output']}")
        else:
            print(f"Multiplier {multiplier}: {result}")
    else:
        print(f"Multiplier {multiplier}: Unexpected result format - {result}")

In [ ]:
run_simple_steering(
    text=["The current date is"],
    model=model,
    tokenizer=tokenizer,
    layer=16,
    multiplier=0,
    vector=direction.detach(),
    max_n_tokens=10,
    save_to=None
)

In [ ]:
run_simple_steering(
    text=["The current date is"],
    model=model,
    tokenizer=tokenizer,
    layer=16,
    multiplier=0,
    vector=direction.detach(),
    max_n_tokens=10,
    save_to=None
)

# CAA

## Let's get some contrast pairs

Let's try an easy direction - positive vs negative sentiment

In [ ]:
GOOD = [
    "The weather is really nice",
    "I'm so happy",
    "This cake is absolutely delicious",
    "I love my friends",
    "I'm feeling great",
    "I'm so excited",
    "This is the best day ever",
    "I really like this gift",
    "Croissants are my favorite",
    "The movie was fantastic",
    "I got a promotion at work",
    "My vacation was amazing",
    "The concert exceeded my expectations",
    "I'm grateful for my family",
    "This book is incredibly engaging",
    "The restaurant service was excellent",
    "I'm proud of my accomplishments",
    "The sunset is breathtakingly beautiful",
    "I passed my exam with flying colors",
    "This coffee tastes perfect",
]

BAD = [
    "The weather is really bad",
    "I'm so sad",
    "This cake is completely inedible",
    "I hate my enemies",
    "I'm feeling awful",
    "I'm so anxious",
    "This is the worst day ever",
    "I dislike this gift",
    "Croissants are disgusting",
    "The movie was terrible",
    "I got fired from work",
    "My vacation was a disaster",
    "The concert was a huge disappointment",
    "I'm frustrated with my family",
    "This book is incredibly boring",
    "The restaurant service was horrible",
    "I'm ashamed of my mistakes",
    "The weather is depressingly gloomy",
    "I failed my exam miserably",
    "This coffee tastes awful",
]

In [ ]:
dataset = [
    (text, True) for text in GOOD
] + [
    (text, False) for text in BAD
]

## Getting the CAA vectors

In [ ]:
if 'vectors' in globals():
    del vectors

vectors = get_caa_vecs(
    labeled_text=dataset,
    model=model,
    tokenizer=tokenizer,
    layers=range(0, 32),
    save_to=None              
)

print(vectors[15])

## Using the CAA vectors

In [ ]:
print(model)
print(tokenizer)

In [ ]:
for layer in range(0,32):
    print(
        run_simple_steering(
            text=["I think that concert is"],
            model=model,
            tokenizer=tokenizer,
            layer=layer,
            multiplier=6,
            vector=vectors[15],
            max_n_tokens=20,
            save_to=None,
        )
    )
    


In [ ]:
run_simple_steering(
    text=["I think that this cat is"],
    model=model,
    tokenizer=tokenizer,
    layer=6,
    multiplier=4,
    vector=vectors[15],
    max_n_tokens=20,
    save_to=None,
)